---
title: 1.6 The Permuted LU-Factorization
subject:  Linear Algebraic Systems
subtitle: Let's mix things up a little
short_title: 1.6 Permuted LU-Factorization
authors:
  - name: Nikolai Matni
    affiliations:
      - Dept. of Electrical and Systems Engineering
      - University of Pennsylvania
    email: nmatni@seas.upenn.edu
license: CC-BY-4.0
keywords: systems of linear equations, permutation matrices, interchanging rows
math:
  '\vv': '\mathbf{#1}'
  '\bm': '\begin{bmatrix}'
  '\em': '\end{bmatrix}'
  '\R': '\mathbb{R}'
---

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/nikolaimatni/ese-2030/HEAD?labpath=/00_Linear_Algebraic_Systems/024-linsys-perms.ipynb)

{doc}`Lecture notes <../lecture_notes/Lecture 02 - Pivots + Permutations, Matrix Inverses (Gauss-Jordan), Transposes, Symmetric Matrices, and General Linear Systems.pdf>`

## Reading
Material related to this page, as well as additional exercises, can be found in ALA Ch. 1.4, LAA Ch 2.5, and ILA Ch. 2.4.  These notes are mostly based on ALA Ch 1.4.

## Learning Objectives

By the end of this page, you should know:
- how to solve linear equations if $A$ is not regular
- what are permutation matrices
- how to use permuted LU factorization

## Interchanging rows

The method of Gaussian Elimination works only if the matrices are regular[^regular]. However, not every matrix is regular as given in the example below.

[^regular]: We'll call a square matrix $A$ _regular_ if Gaussian Elimination successfully reduces $A$ to the upper triangular form $U$ with all nonzero pivots on the diagonal.  If this fails to happen, i.e., if a pivot appearing on the diagonal is zero, then the matrix is not regular.

\begin{eqnarray}
\label{non_regular}
2y + z & = 2,\\
2x +6y + z & = 7,\\
x+y+4z & =3,
\end{eqnarray}

The augmented coefficient matrix for the above set of equations is

\begin{equation}
\label{augm}
\left[ \begin{array}{ccc|c} 0 & 2 & 1 & 2\\ 2& 6 & 1 & 7\\ 1 & 1 & 4 & 3 \end{array}\right].
\end{equation}

In the above example, the entry $(1,1)$ is 0, which is the first diagonal element, but cannot serve as a pivot. The "problem" is because $x$ does not appear in the first equation, but this is actually a bonus because we already have an equation that has only two variables in it. Hence, we need to eliminate $x$ from only one of the other two equations. Let us interchange the first two rows of [](#augm):

\begin{equation}
\label{augm_swap}
\left[ \begin{array}{ccc|c}  2& 6 & 1 & 7\\ 0 & 2 & 1 & 2\\ 1 & 1 & 4 & 3 \end{array}\right],
\end{equation}

which clearly does not change the solution set, but now we have a pivot at $(1,1)$ and $(2,2)$. Interchanging the first and second row is equivalent to swapping the first and second equation. Now, we can proceed as in Gaussian Elimination to zero out $(3, 1)$ and $(3, 2)$ using [elementary row operations](./023-linsys-gauuss.ipynb#rowop1) to get  

\begin{equation}
\label{augm_swap}
\left[ \begin{array}{c|c}  U &  \textbf{c} \end{array}\right] = 
\left[ \begin{array}{ccc|c}  2& 6 & 1 & 7\\ 0 & 2 & 1 & 2\\ 0 & 0 & \frac{9}{2} & \frac{3}{2} \end{array}\right].
\end{equation}

The pivots are $2, 2, \frac{9}{2}$ and solving via back substitution yields the solution $x=\frac{5}{6}, y = \frac{5}{6}, z = \frac{1}{3}$. We observe the swapping operation as another elementary row operation, in addition to [type 1](./023-linsys-gauuss.ipynb#rowop1).

```{prf:observation} Elementary Row Operation \#2
:label: rowop2
Interchanging two rows of the augmented matrix $M$ is the equivalent of swapping two equations in the system of linear equations it defines. As such, this does not change the solution set and leads to an equivalent augmented matrix.
```

So far, we have worked with matrices than can be reduced to upper triangular form with all non-zero diagonals, which is formally defined below.

```{prf:definition} Nonsingular
:label: nonsingular
A square matrix is called _nonsingular_ if it can be reduced to upper triangular form with all non-zero elements on the diagonal — the pivots — by elementary row operations of types [1](./023-linsys-gauuss.ipynb#rowop1) and [2](#rowop2).
```

In contrast, a singular square matrix cannot be reduced to such upper triangular form
by such row operations, because at some stage in the elimination procedure the diagonal
entry and all the entries below it are zero. 

```{warning}
:label: nonsingular_regular
Every regular matrix is nonsingular, but, as we just saw, not every nonsingular matrix is regular.
```

```{prf:theorem} Uniqueness of solution
:label: uniqueness
A linear system $A \textbf{x} = \textbf{b}$ has a unique solution for every choice of right-hand side $b$ if and only if its coeﬃcient matrix $A$ is square and [nonsingular](#nonsingular).
```
We are able to prove the “if” part of this theorem, since nonsingularity implies reduction
to an equivalent upper triangular form that has the same solutions as the original system.
The unique solution to the system is then found by back substitution. The “only if” part will be proved later.

## Small Pivots cause numerical issues

Consider the example

\begin{eqnarray}
\label{small_pivot}
0.01x +1.6y & = 32.1,\\
x+0.6y & = 22,
\end{eqnarray}

where, the exact solution is $x=10, y=20$ (verify for yourself). Let us assume we have a calculator that only retains three digits. The actual augmented matrix is given below after making it upper triangular.

\begin{equation} 
\left[ \begin{array}{cc|c}  .01 & 1.6 & 32.1\\ 1 & .6 & 22 \end{array}\right] \leftrightarrow 
\left[ \begin{array}{cc|c}  .01 & 1.6 & 32.1\\ 0 & -159.4 & -3188 \end{array}\right].
\end{equation}

Since the calculator only retains three digits, the matrix computed by the calculator is
\begin{equation} 
\label{small_pivot_augm} 
\left[ \begin{array}{cc|c}  .01 & 1.6 & 32.1\\ 0 & -159 & -319 \end{array}\right].
\end{equation}

Solving by back substitution using [](#small_pivot_augm) is $y = \frac{-3190}{159} \approx 20.1 \Rightarrow x \approx -10$. Hence, a small error in $y$, i.e,  0.1, produces a large error in $x$! The problem is because of the small pivot $0.01$ at $(1,1)$. The small error in $y$ is scaled up when evaluating the first equation, where we have to divide the exression by the small pivot $0.01$. If we interchanged the rows so that the pivot is $1$ and proceed from there via Gauss Elimination, the approximate solution is ${y \approx 20.1, x \approx 9.9}$.

```{tip}
:label: interchange_small_pivot
Small pivots cause issues in numerical computations. Use row interchanges to avoid small pivots.
```

```{note}
:label: finite_digits
Computers have much higher capacity to retain digits, but still finite. Similar numerical issues arise in computers as well. 
```

## Permutation matrices

Similar to how [type 1 operations](./023-linsys-gauuss.ipynb#rowop1) are represented using [elementary matrices](./023a-linsys-LU.ipynb#elementary), we can represent [type 2 operations](rowop2) using _permutation matrices_. 

```{prf:definition} Permutation matrices
:label: perm_matrix
A permutation matrix is a matrix obtained from the identity matrix by _any_ combination of row interchanges.
```

For example, to swap rows 1 and 2 of the matrix $A$, we left multiply $A$ by $P$ as follows.

\begin{equation} 
\label{perm_matrix} 
A = \begin{bmatrix}
1 & 2 & 3 \\
4 & 5 & 6 \\
7 & 8 & 9
\end{bmatrix}, P = \begin{bmatrix}
0 & 1 & 0 \\
1 & 0 & 0 \\
0 & 0 & 1
\end{bmatrix} \Rightarrow PA = \begin{bmatrix}
4 & 5 & 6 \\
1 & 2 & 3 \\
7 & 8 & 9
\end{bmatrix}
\end{equation}

```{note} Also a Permutation matrix
:label: perm_matrix_alt
A matrix $P$ is a permutation matrix if and only if each row of $P$ contains all $0$ entries except for a single $1$, and, in addition, each column of $P$ also contains all $0$ entries except for a single $1$
```

:::{important} Properties
1. If $P_1, P_2$ are permutation matrices, then their products $P_1P_2, P_2P_1$ are also permutation matrices.
2. In general, permutation matrices do not commute $P_1P_2 \neq P_2P_1$, since the order of swapping matters.
3. If $P_1$ is a permutation matrix that swaps exactly one row with another, then $P_1P_1 = I$, because the inverse operation of interchanging two rows is the same original operation. This is **not true** for general permutation matrices that encode multiple swaps. 
:::

## Permuted LU-factorization

From [](#nonsingular), every nonsingular matrix $A$ can be reduced to upper triangular form using [type 1 operations](./023-linsys-gauuss.ipynb#rowop1) and [type 2 operations](rowop2). If we perform all the permutations first using a permutation matrix $P$, then we obtain a [regular matrix](./023a-linsys-LU.ipynb#regular_matrix) $PA$ that admits an $LU$ factorization. Hence we can define the _permuted LU factorization_ of a nonsingular matrix $A$ as
\begin{equation} 
\label{perm_LU} 
PA = LU
\end{equation}

Given a permuted LU factorization of the matrix $A$, we can solve the system of equations by first doing forward substitution with $L\textbf{z}= P\textbf{b}$ and back substitution with $U\textbf{x} = \textbf{z}$. 

````{exercise} Permuting a linear system
:label: linsys-perms-ex1

Consider the following linear system:

\begin{align*}
    x + 2y + z = -2\\
    2x + 4y - z = 12\\
    -x + y - z = 5
\end{align*}

Show that the coefficient matrix

\begin{align*}
    A = \bm 
        1 & 2 & 1 \\
        2 & 4 & -1 \\
        -1 & 1 & -1
    \em
\end{align*}

is irregular. 

Then, given the permutation matrix

\begin{align*}
    P = \bm
        1&0&0\\
        0&0&1\\
        0&1&0
    \em
\end{align*}

show that the permuted coefficient matrix $PA$ is regular. 

Finally, solve the permuted system $PA = P \mathbf b$ by first computing the $LU$ factorization of $PA$, and then using forward and back substitution with $PA\mathbf x = LU\mathbf x = P\mathbf b$.

:::{hint} Click me for a hint!
:class: dropdown
Remember, a matrix is regular if and only if it has an $LU$ factorization. What happens when we run the $LU$ factorization algorithm on $A$?

:::
```{solution} linsys-perms-ex1
:class: dropdown

First, let's prove that $A$ is irregular. Let's run the $LU$ factorization algorithm on $A$ (which is really just Gaussian elimination). With $1$ as our first pivot, we eliminate column 1:

\begin{align*}
    \underbrace{\bm 1&2&1\\2&4&-1\\-1&1&-1 \em}_A &= \bm 1&0&0\\0&1&0\\0&0&1\em \bm 1&2&1\\2&4&-1\\-1&1&-1 \em\\
    &= \bm 1&0&0\\0&1&0\\0&0&1\em\left(\bm 1&0&0\\2&1&0\\-1&0&1 \em \bm 1&0&0\\-2&1&0\\1&0&1 \em\right) \bm 1&2&1\\2&4&-1\\-1&1&-1 \em\\
    &=\left(\bm 1&0&0\\0&1&0\\0&0&1\em\bm 1&0&0\\2&1&0\\-1&0&1 \em \right)  \left(\bm 1&0&0\\-2&1&0\\1&0&1 \em \bm 1&2&1\\2&4&-1\\-1&1&-1 \em\right)\\
    &=\bm 1&0&0\\2&1&0\\-1&0&1 \em \bm 1&2&1\\0&0&-3\\0&3&0 \em
\end{align*}

Uh oh! Our second pivot (the element at position (2, 2) of the rightmost matrix in the bottom expression) is now zero, so the $LU$ factorization algorithm fails. This means that $A$ doesn't have an $LU$ factorization, so $A$ is irregular.

The permuted matrix $PA$ is

\begin{align*}
    PA = \underbrace{\bm
        1&0&0\\
        0&0&1\\
        0&1&0
    \em}_P \underbrace{\bm 1&2&1\\2&4&-1\\-1&1&-1 \em}_A = \bm 1&2&1\\-1&1&-1\\2&4&-1 \em
\end{align*}

Now, let's run the $LU$ factorization algorithm on $PA$. With $1$ as our first pivot, we eliminate column $1$:

\begin{align*}
    \underbrace{\bm 1&2&1\\-1&1&-1\\2&4&-1 \em}_{PA} &= \bm 1&0&0\\0&1&0\\0&0&1\em \bm 1&2&1\\-1&1&-1\\2&4&-1 \em\\
    &= \bm 1&0&0\\0&1&0\\0&0&1\em\left(\bm 1&0&0\\-1&1&0\\2&0&1 \em \bm 1&0&0\\1&1&0\\-2&0&1 \em\right) \bm 1&2&1\\-1&1&-1\\2&4&-1 \em\\
    &= \left(\bm 1&0&0\\0&1&0\\0&0&1\em\bm  1&0&0\\-1&1&0\\2&0&1 \em\right) \left(\bm 1&0&0\\1&1&0\\-2&0&1 \em\bm 1&2&1\\-1&1&-1\\2&4&-1 \em\right)\\
    &= \underbrace{\bm 1&0&0\\-1&1&0\\2&0&1 \em}_L \underbrace{\bm 1&2&1\\0&3&0\\0&0&-3 \em}_U
\end{align*}

For this exercise, we had a small 3x3 system, so that's it for the $LU$ factorization algorithm! 

The last step is to solve with forwardsubstition and backsubstitution on the system:

\begin{align*}
    \underbrace{\bm 1&0&0\\-1&1&0\\2&0&1 \em}_L \underbrace{\bm 1&2&1\\0&3&0\\0&0&-3 \em}_U \underbrace{\bm x\\y\\z \em}_{\mathbf x} = \underbrace{\bm -2\\5\\12 \em}_{P \mathbf b}
\end{align*}

And we get:

\begin{align*}
    U\mathbf x = \bm -2\\3\\18 \em, \quad \mathbf x = \bm 2\\ 1\\ -6\em
\end{align*}

```
````

```{prf:example} Simultaneously finding the P, L, and U matrices

So far, we've learned that *every* nonsingular square matrix $A$ has at least one factorization of the form $PA = LU$, where $P$ is a permutation matrix and $L, U$ are lower and upper triangular, respectively. 

In [](#linsys-perms-ex1), you were given the permutation $P$ beforehand. But what if don't have prior knowledge of $P$?

It turns out that, given a square nonsingular matrix $A$, we can compute the $P, L, U$ matrices simultaneously. In this example, we'll demonstrate a procedure for computing $P, L, U$ simultaneously using the same system as [](#linsys-perms-ex1). It's basically just Gaussian elimination with row swaps!

First, let's write out the obvious equality 

\begin{align*}
    \underbrace{\bm 1&0&0\\0&1&0\\0&0&1 \em}_{"P"}\underbrace{\bm 1&2&1\\2&4&-1\\-1&1&-1 \em}_A = \underbrace{\bm 1&0&0\\0&1&0\\0&0&1 \em}_{"L"} \underbrace{\bm 1&2&1\\2&4&-1\\-1&1&-1 \em}_{"U"}
\end{align*}

You'll note that the RHS has 2 matrices named $"L"$ and $"U"$. The idea of the procedure is to use some matrix algebra rules to transform $"L"$ into a lower tringular matrix and $"U"$ into an upper triangular matrix. In the process, we are going to build up the permutation matrix $"P"$.

Starting off, let's use $1$ as our first pivot to eliminate column 1:

\begin{align*}
    \underbrace{\bm 1&0&0\\0&1&0\\0&0&1 \em}_{"P"}\underbrace{\bm 1&2&1\\2&4&-1\\-1&1&-1 \em}_A &= \underbrace{\bm 1&0&0\\0&1&0\\0&0&1\em}_{"L"} \underbrace{\bm 1&2&1\\2&4&-1\\-1&1&-1 \em}_{"U"}\\
    &= \bm 1&0&0\\0&1&0\\0&0&1\em\left(\bm 1&0&0\\2&1&0\\-1&0&1 \em \bm 1&0&0\\-2&1&0\\1&0&1 \em\right) \bm 1&2&1\\2&4&-1\\-1&1&-1 \em\\
    &=\left(\bm 1&0&0\\0&1&0\\0&0&1\em\bm 1&0&0\\2&1&0\\-1&0&1 \em \right)  \left(\bm 1&0&0\\-2&1&0\\1&0&1 \em \bm 1&2&1\\2&4&-1\\-1&1&-1 \em\right)\\
    &=\underbrace{\bm 1&0&0\\2&1&0\\-1&0&1 \em}_{"L"} \underbrace{\bm 1&2&1\\0&0&-3\\0&3&0 \em}_{"U"}
\end{align*}

Just like in [](#linsys-perms-ex1), we run into the same issue of a zero pivot. How can we fix this? Recall that for any permutation matrix $P$ swapping 2 rows, $PP = I$ (In this case, we want $P$ to be the permutation matrix swapping rows 2 and 3 of $"U"$). We can cleverly insert this expression in the middle of $"L"$ and $"U"$, and use associativity of matrix multiplication to get:

\begin{align*}
    \underbrace{\bm 1&0&0\\0&1&0\\0&0&1 \em}_{"P"}\underbrace{\bm 1&2&1\\2&4&-1\\-1&1&-1 \em}_A &=\underbrace{\bm 1&0&0\\2&1&0\\-1&0&1 \em}_{"L"} \underbrace{\bm 1&2&1\\0&0&-3\\0&3&0 \em}_{"U"}\\
    &= \underbrace{\bm 1&0&0\\2&1&0\\-1&0&1 \em}_{"L"} \left( \bm 1&0&0\\0&0&1\\0&1&0 \em \bm 1&0&0\\0&0&1\\0&1&0 \em \right) \underbrace{\bm 1&2&1\\0&0&-3\\0&3&0 \em}_{"U"}\\
    &= \left(\underbrace{\bm 1&0&0\\2&1&0\\-1&0&1 \em}_{"L"} \bm 1&0&0\\0&0&1\\0&1&0 \em  \right)\left(\bm 1&0&0\\0&0&1\\0&1&0 \em  \underbrace{\bm 1&2&1\\0&0&-3\\0&3&0 \em}_{"U"}\right)\\
    &= \underbrace{\bm 1&0&0\\2&0&1\\-1&1&0 \em}_{"L"} \underbrace{\bm 1&2&1\\0&3&0\\0&0&-3 \em}_{"U"}
\end{align*}

We're almost there! However, our "L" matrix isn't lower triangular yet! Not to worry, we can fix this with a single row swap: 
\begin{align*}
    &\underbrace{\bm 1&0&0\\0&1&0\\0&0&1 \em}_{"P"}\underbrace{\bm 1&2&1\\2&4&-1\\-1&1&-1 \em}_A =\underbrace{\bm 1&0&0\\2&0&1\\-1&1&0 \em}_{"L"} \underbrace{\bm 1&2&1\\0&3&0\\0&0&-3 \em}_{"U"}\\
    &\underset{\text{(implies)}}{\implies} \bm 1&0&0\\0&0&1\\0&1&0 \em \underbrace{\bm 1&0&0\\0&1&0\\0&0&1 \em}_{"P"}\underbrace{\bm 1&2&1\\2&4&-1\\-1&1&-1 \em}_A = \bm 1&0&0\\0&0&1\\0&1&0 \em\underbrace{\bm 1&0&0\\2&0&1\\-1&1&0 \em}_{"L"} \underbrace{\bm 1&2&1\\0&3&0\\0&0&-3 \em}_{"U"}\\
    &\implies \underbrace{\bm 1&0&0\\0&0&1\\0&1&0 \em}_{"P"}\underbrace{\bm 1&2&1\\2&4&-1\\-1&1&-1 \em}_A = \underbrace{\bm 1&0&0\\-1&1&0\\2&0&1 \em}_{"L"} \underbrace{\bm 1&2&1\\0&3&0\\0&0&-3 \em}_{"U"}
\end{align*}

(An astute reader might notice that $"L"$ was unchanged from multiplying it from both the left and right by $P$ (i.e., $"L" = P \times "L" \times "P"$) where $P$ was the row swap we defined above. This is no coincidence, and stems from the fact that $"L"$ has a particular structure which we will discuss later.)

And we're done! But this example was only for a small 3x3 system, where we only had to make one row swap. Let's try to distill the essence of what we just did into a more general procedure to compute the $PA = LU$ factorization of any square nonsingular matrix (this same procedure will generalize to nonsquare matrices as well).

* First, initialize your matrices:

  * $"P"$ starts off as the identity matrix.

  * $"L"$ starts off as the identity matrix.

  * $"U"$ starts off as the input coefficient matrix $A$.

* Second, we are going to perform a series of steps (Step $1$, Step $2$, ..., Step $n$) such that at end of Step $k$, for $k = 1, 2, ..., n$, the following *invariants* are satisfied:

  * $"P"\times A = "L" \times "U"$.

  * $"P"$ is a permutation matrix.
 
  * $L$ is lower triangular.

  * The *trailing* (bottomright-most) $(n - k) \times (n - k)$ submatrix of $"L"$ is the identity matrix and only has zeros above it in $L$.

  * The *leading* (topleft-most) $k \times k$ submatrix of $U$ is upper triangular and only has zeros below it in $"U"$.

  If we do the following, we can always preserve these invariants. At Step $k$,

  * If the pivot at position $(k, k)$ is nonzero, then continue onto the next step. Otherwise, swap it with an element *below it*, and *left multiply* $"P"$ by the permutation matrix corresponding to the rows that were swapped.
  
  * Use said pivot at position $(k, k)$ to eliminate the entries *below it* by *left multiplying* $"U"$ by some elementary matrices, and *right multiplying* "L" by the inverse of those elementary matrices.

* Finally, by our invariants, at the end of Step $n$: $"P"$, $"L"$, $"U"$ will be the $PA = LU$ factorization of $A$.

In this course, we don't place an emphasis on the gritty details behind this procedure, so we won't prove it's correctness here. But still, try to convince yourself that this procedure works for square, nonsingular matrices!

```

In [1]:
# Permuted LU factorization

import numpy as np
from scipy.linalg import lu

A = np.array([[0, 2, 1],
              [2, 6, 1],
              [1, 1, 4]])
P, L, U = lu(A)
print("P: \n", P, "\nL: \n", L, "\nU: \n", U)

P: 
 [[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]] 
L: 
 [[ 1.   0.   0. ]
 [ 0.   1.   0. ]
 [ 0.5 -1.   1. ]] 
U: 
 [[2.  6.  1. ]
 [0.  2.  1. ]
 [0.  0.  4.5]]


[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/nikolaimatni/ese-2030/HEAD?labpath=/00_Linear_Algebraic_Systems/024-linsys-perms.ipynb)
